In [2]:
import os
import skimage.io
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from glob import glob
import sys
sys.path.append('../input/sources')
from sync_batchnorm import SynchronizedBatchNorm1d
from preprocess_model import load_models
from preprocess_load_data import inference_PANDADataset
from torch.utils.data import DataLoader
data_dir = '../input/prostate-cancer-grade-assessment/'
df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))

In [7]:
df=pd.read_csv('../cropped_img/labels_512.csv')

In [8]:
df

,image_id,label
0,e89fd05c02c2297653d93f9bdaa71961_0.png,3
1,9b238129762edd83f472e5b3abf9e639_9.png,0
2,3bb8426f5536b507843d3c99ab0a5e8e_15.png,4
3,ef8cb1391e62cb6721eddf6e3495087d_31.png,2
4,a954e34aa74d5c6646a52e605263f187_4.png,4
...,...,...
82206,97845cbd34d7c2f9ea7d76fa931f5412_12.png,4
82207,64d853ef2f8fd85e3a9b09d3791780e7_4.png,2
82208,5d656ef89286db380f879cb6bfadd731_10.png,4
82209,a21ea7caca40bc45f043ed27ae2b573d_11.png,0


In [1]:
import os
import skimage.io
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from glob import glob
import sys
sys.path.append('../input/sources')
from sync_batchnorm import SynchronizedBatchNorm1d
from preprocess_model import load_models
from preprocess_load_data import inference_PANDADataset
from torch.utils.data import DataLoader



crop_size=512
image_size = 1024

def inference_gleason():

    data_dir = '../input/prostate-cancer-grade-assessment/'
    df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
    df_train=df_train[df_train['data_provider']=='karolinska']

    image_folder = os.path.join(data_dir, 'train_images/')
    model_dir='../result_weight/'

    batch_size = 12
    num_workers = 12
    out_dim = 6

    backborn='efficientnet-b4'

    device = torch.device('cuda')

    img_list=[]
    for img_id in tqdm(df['image_id'].values):
        img=skimage.io.MultiImage(os.path.join(image_folder,img_id)+'.tiff')[1]
        for i in range(img.shape[0]//crop_size+1):
            for j in range(img.shape[1]//crop_size+1):
                img_list.append(img_id+'_'+str(i)+'_'+str(j))

    df_cropped=pd.DataFrame({'image_id':img_list})

    model_files = [
        'efficient_base_1024to1024_without_marks_best_fold0.pth'
    ]

    models = load_models(model_files,model_dir,out_dim,device,backborn)

    dataset = inference_PANDADataset(df_cropped, image_size,crop_size,image_folder)
    loader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False)

    LOGITS = []
    PREDS=[]
    with torch.no_grad():
        for data in tqdm(loader):
            data = data.to(device)
            logits = models[0](data)
            LOGITS.append(logits)
            pred=logits.argmax(1).detach()
            PREDS.append(pred)

    LOGITS = torch.cat(LOGITS).cpu().numpy()
    PREDS = torch.cat(PREDS).cpu().numpy()

    imgid=[imgid.split('_')[0] for imgid in df_cropped['image_id'].values]
    idx1=[imgid.split('_')[1] for imgid in df_cropped['image_id'].values]
    idx2=[imgid.split('_')[2] for imgid in df_cropped['image_id'].values]

    tmp=pd.DataFrame({'image_id':imgid,
                'idx1':np.array(idx1).astype(int),
                'idx2':np.array(idx2).astype(int),
                'gleason':PREDS})

    return tmp

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
df=inference_gleason()
df.to_csv('../result/inference_gleason'+str(image_size)+'.csv')